In [3]:
import requests
import random
import string

In [24]:
res = requests.get("https://wordle.votee.dev:8000/word/sieges?guess=sicks")

In [30]:
res.content

b'Guess must be the same length as the word'

In [72]:
BASE_URL = "https://wordle.votee.dev:8000/"
LIMIT_ATTEMPTS = 100

class WordleSolver:
    def __init__(self, size):
        self.size = size
        self.correct = [None] * self.size
        self.present = {}
        self.absent = set()
        self.attempts = []
    
    def process_feedback(self, feedback):
        new_absent = set()
        for res in feedback:
            char = res['guess'].lower()
            slot = res['slot']
            result =res['result']

            if result == 'correct':
                self.correct[slot] = char
                #if char in self.present:
                #    del self.present[char]
            
            elif result == 'present':
                if char not in self.present:
                    self.present[char] = set()
                self.present[char].add(slot)
                if char in self.absent:
                    self.absent.remove(char)
            
            elif result == 'absent':
                if all(char != c for c in self.correct if c) and char not in self.present:
                    new_absent.add(char)
            
        self.absent.update(new_absent)

    def generate_guess(self):
        # Start with the correct letters
        guess = list(self.correct)
        remaining_slots = [i for i, c in enumerate(guess) if c is None]

        # Shuffle for avoiding patterns
        random.shuffle(remaining_slots)
        present_chars = [c for c in self.present if c not in guess]

        # Place present characters
        for char in present_chars:
            allowed_slots = [s for s in remaining_slots if s not in self.present.get(char, set())]
            if allowed_slots:
                slot = random.choice(allowed_slots)
                guess[slot] = char
                remaining_slots.remove(slot)

        # Fill remaining slots with valid alphabets
        for slot in remaining_slots:
            valid_chars = [c for c in string.ascii_lowercase if c not in self.absent and c not in guess]
            try:
                guess[slot] = random.choice(valid_chars) if valid_chars else random.choice([i for i in self.correct if i is not None])
            except:
                guess[slot] = random.choice(valid_chars) if valid_chars else random.choice(string.ascii_lowercase)
        return ''.join(guess)

    def solve_daily(self):
        attempts = 0
        while attempts < LIMIT_ATTEMPTS:
            attempts += 1
            guess = self.generate_guess().lower()

            # Skip duplicate guesses
            if guess in self.attempts:
                continue

            print(f"Attempt {attempts}: {guess}")
            
            response = requests.get(f"{BASE_URL}/daily", params={"guess": guess, "size":self.size})

            if response.status_code == 200:
                feedback = response.json()
                self.attempts.append(guess)
                self.process_feedback(feedback)

                if all(c is not None for c in self.correct):
                    print(f"\nSolved in {attempts} attempts!")
                    print(f"Final Answer: {''.join(self.correct).lower()}")
                    return True
            
            else:
                print(f"Error Status Code: {response.status_code}")
                print(f"Error detail: {response.content}")
                break
        
        print(f"\nFailed to solve within {attempts} attempts")
        return False


    def solve_random(self):
        seed = random.randint(0,1000)
        attempts = 0
        while attempts < LIMIT_ATTEMPTS:
            attempts += 1
            guess = self.generate_guess().lower()

            # Skip duplicate guesses
            if guess in self.attempts:
                continue

            print(f"Attempt {attempts}: {guess}")
            
            response = requests.get(f"{BASE_URL}/random", params={"guess": guess, "size":self.size, "seed":seed})

            if response.status_code == 200:
                feedback = response.json()
                self.attempts.append(guess)
                self.process_feedback(feedback)

                if all(c is not None for c in self.correct):
                    print(f"\nSolved in {attempts} attempts!")
                    print(f"Final Answer: {''.join(self.correct).lower()}")
                    return True
            
            else:
                print(f"Error Status Code: {response.status_code}")
                print(f"Error detail: {response.content}")
                break
        
        print(f"\nFailed to solve within {attempts} attempts")
        return False


    def solve_word(self, word):
        attempts = 0
        while attempts < LIMIT_ATTEMPTS:
            attempts += 1
            guess = self.generate_guess().lower()

            # Skip duplicate guesses
            if guess in self.attempts:
                continue

            print(f"Attempt {attempts}: {guess}")
            response = requests.get(f"{BASE_URL}/word/{word}", params={"guess": guess, "size":self.size})

            if response.status_code == 200:
                feedback = response.json()
                self.attempts.append(guess)
                self.process_feedback(feedback)

                if all(c is not None for c in self.correct):
                    print(f"\nSolved in {attempts} attempts!")
                    print(f"Final Answer: {''.join(self.correct).lower()}")
                    return True
            
            else:
                print(f"Error Status Code: {response.status_code}")
                print(f"Error detail: {response.content}")
                break
        
        print(f"\nFailed to solve within {attempts} attempts")
        return False



In [ ]:
solver = WordleSolver(size=16)
solver.solve_random()

Attempt 1: smjiconryhpfbazt
Attempt 2: xgkcioehydwpqvsl
Attempt 3: cyuccoohyogsiylp
Attempt 4: gooclohhycohsipc
Attempt 5: ichchohhypsclgyc
Attempt 6: ylpchoohyhlilygs
Attempt 7: lihchoshyhoolpgo
Attempt 8: plichoyhyscolhgg
Attempt 9: pghchochysiolhgo
Attempt 10: pipchoshysiologh
Attempt 11: pcgchoihysiologg
Attempt 12: phycholhysiologo
Attempt 13: ppychoyhysiologo
Attempt 14: piychohhysiologo
Attempt 15: poychohhysiologo
Attempt 16: piychoohysiologo
Attempt 17: poychoyhysiology
Attempt 18: psychoohysiology
Attempt 19: psychoghysiology
Attempt 20: psychohhysiology
Attempt 21: psychophysiology

Solved in 21 attempts!
Final Answer: psychophysiology


True

In [71]:
solver = WordleSolver(size=14)
solver.solve_random()

Attempt 1: veiwtrgpscbxom
Attempt 2: fcpqiaherjosln
Attempt 3: cherpasldikyuo
Attempt 4: epcosalzyrihda
Attempt 5: yadhsaerlocidp
Attempt 6: orylsaadchpide
Attempt 7: rohcsayipelide
Attempt 8: horlsacaapyide
Attempt 9: pooysacehlride
Attempt 10: polysacdhoride
Attempt 11: polysaclheride
Attempt 12: polysacphlride
Attempt 13: polysacahhride
Attempt 14: polysacdhsride
Attempt 15: polysacshhride
Attempt 16: polysacrhlride
Attempt 17: polysacdhpride
Attempt 18: polysacrhdride
Attempt 19: polysacoheride
Attempt 20: polysacdhiride
Attempt 21: polysacphdride
Attempt 22: polysacshlride
Attempt 23: polysacdharide
Attempt 24: polysacsharide
Attempt 25: polysacyharide
Attempt 26: polysacrharide
Attempt 27: polysacaharide
Attempt 31: polysaciharide
Attempt 32: polysaclharide
Attempt 36: polysaceharide
Attempt 42: polysachharide
Attempt 48: polysaccharide

Solved in 48 attempts!
Final Answer: polysaccharide


True

In [75]:
solver = WordleSolver(size=5)
solver.solve_word("pieces")

Attempt 1: jiafb
Error Status Code: 400
Error detail: b'Guess must be the same length as the word'

Failed to solve within 1 attempts


False